# Exploration of SG Bus Fares using Tableau

Data from [data.gov.sg](https://data.gov.sg/) on the following 2 kinds of Bus Services:

1. [Trunk Bus Services](https://data.gov.sg/dataset/fare-structure-for-trunk-bus-services)
2. [Express Bus Services](https://data.gov.sg/dataset/fare-for-express-bus-services)

The data was then briefly cleaned here, processed on Tableau (where the tables were pivoted, and the columns were split so that the "Passenger Type" and "Payment Type" were in different columns), and then brought back here for a final round of cleaning and touching up before being visualized in Tableau.

To start with, we'll import the libraries needed.

In [1]:
import numpy as np
import pandas as pd

## Data Cleaning

The two dataframes from Tableau were read in as `trunk` and `express`. The `km` unit from the column value was removed and placed in the column name.

In [2]:
trunk = pd.read_csv('fares-for-trunk-bus-service-effective-from-29-december-2018.csv')

trunk['distance'] = trunk['distance'].str.replace("Up to", "").str.replace("Over", "").str.replace("km", "")

In [3]:
trunk.iloc[0,0] = "3.2 or less"
trunk.iloc[38,0] = "40.2 or more"

In [4]:
trunk.rename({'distance':'distance (km)'}, axis =1, inplace = True)

trunk.head()

,distance (km),adult_card_fare_per_ride,adult_cash_fare_per_ride,senior_citizen_card_fare_per_ride,senior_citizen_cash_fare_per_ride,student_card_fare_per_ride,student_cash_fare_per_ride,workfare_transport_concession_card_fare_per_ride,workfare_transport_concession_cash_fare_per_ride,persons_with_disabilities_card_fare_per_ride,persons_with_disabilities_cash_fare_per_ride
0,3.2 or less,83,150,55,100,38,65,64,150,55,100
1,3.3 - 4.2,93,170,62,100,43,65,72,170,62,100
2,4.3 - 5.2,103,170,69,100,48,65,80,170,69,100
3,5.3 - 6.2,113,170,76,100,53,65,88,170,76,100
4,6.3 - 7.2,122,190,82,130,56,85,95,190,82,130


In [5]:
express = pd.read_csv('fares-for-express-bus-services-effective-from-29-december-2018.csv')
express['distance'] = express['distance'].str.replace("Up to", "").str.replace("Over", "").str.replace("km", "")

express.iloc[0,0] = "3.2 or less"
express.iloc[38,0] = "40.2 or more"

express.rename({'distance':'distance (km)'}, axis =1, inplace = True)

express.head()

,distance (km),cash_fare_per_ride,adult_card_fare_per_ride,senior_citizen_card_fare_per_ride,student_card_fare_per_ride,workfare_transport_concession_card_fare_per_ride,persons_with_disabilities_card_fare_per_ride
0,3.2 or less,230,143,100,68,114,100
1,3.3 - 4.2,230,153,107,73,122,107
2,4.3 - 5.2,230,163,114,78,130,114
3,5.3 - 6.2,230,173,121,83,138,121
4,6.3 - 7.2,230,182,127,86,145,127


In [6]:
# trunk.to_csv('trunk')
# express.to_csv('express')

## Pivoting in Tableau

The dataframes were pivoted in Tableau, and then brought back here for their final cleaning.

## Final Round of Data Cleaning

### Cleaning the dataframe for Trunk Buses

In [7]:
trunks2 = pd.read_csv('trunk.txt (Multiple Connections).csv')

trunks2.head()

,Number of Records,Passenger Type,Payment Type,Fare,Distance (Km)
0,1,Adult,Adult Card Fare Per Ride,83,3.2 or less
1,1,Adult,Adult Card Fare Per Ride,93,3.3 - 4.2
2,1,Adult,Adult Card Fare Per Ride,103,4.3 - 5.2
3,1,Adult,Adult Card Fare Per Ride,113,5.3 - 6.2
4,1,Adult,Adult Card Fare Per Ride,122,6.3 - 7.2


The unnecessary Column, `Number of Records`, was dropped.

In [8]:
trunks2 = trunks2.drop('Number of Records', axis=1)

trunks2.head()

,Passenger Type,Payment Type,Fare,Distance (Km)
0,Adult,Adult Card Fare Per Ride,83,3.2 or less
1,Adult,Adult Card Fare Per Ride,93,3.3 - 4.2
2,Adult,Adult Card Fare Per Ride,103,4.3 - 5.2
3,Adult,Adult Card Fare Per Ride,113,5.3 - 6.2
4,Adult,Adult Card Fare Per Ride,122,6.3 - 7.2


In [9]:
trunks2['Passenger Type'].unique()

array(['Adult', 'Persons', 'Senior', 'Student', 'Workfare'], dtype=object)

A dictionary to base the renaming of column walues on was constructed based on the Unique values in the `Passenger Type` column. The point of the re-naming was to make the column values in `Passenger Type` more suitable.

In [25]:
rename_dict = {'Adult': 'Adult',
'Persons': 'Disabled',
'Senior': 'Senior',
'Student': 'Student',
'Workfare': 'Worker',
              'Disabled': 'Disabled',
              'Worker': 'Worker'}

trunks2['Passenger Type'] = trunks2['Passenger Type'].map(rename_dict)

print(trunks2.head())

print(trunks2.tail())

trunks2['Passenger Type'].value_counts()

  Passenger Type Payment Type  Fare Distance (Km)  Transport
0          Adult         Card    83   3.2 or less  Trunk Bus
1          Adult         Card    93    3.3  - 4.2  Trunk Bus
2          Adult         Card   103    4.3  - 5.2  Trunk Bus
3          Adult         Card   113    5.3  - 6.2  Trunk Bus
4          Adult         Card   122    6.3  - 7.2  Trunk Bus
    Passenger Type Payment Type  Fare Distance (Km)  Transport
385         Worker         Cash   260  36.3  - 37.2  Trunk Bus
386         Worker         Cash   260  37.3  - 38.2  Trunk Bus
387         Worker         Cash   260  38.3  - 39.2  Trunk Bus
388         Worker         Cash   260  39.3  - 40.2  Trunk Bus
389         Worker         Cash   260  40.2 or more  Trunk Bus


Student     78
Adult       78
Worker      78
Senior      78
Disabled    78
Name: Passenger Type, dtype: int64

We see from the value counts above that the cleaning was done satisfactorily. Next, we'll rename the values in the `Payment Type` column

In [11]:
trunks2['Payment Type'].unique()

array(['Adult Card Fare Per Ride',
       'Persons With Disabilities Card Fare Per Ride',
       'Senior Citizen Card Fare Per Ride', 'Student Card Fare Per Ride',
       'Workfare Transport Concession Card Fare Per Ride',
       'Adult Cash Fare Per Ride',
       'Persons With Disabilities Cash Fare Per Ride',
       'Senior Citizen Cash Fare Per Ride', 'Student Cash Fare Per Ride',
       'Workfare Transport Concession Cash Fare Per Ride'], dtype=object)

The `Payment Type` column was also renamed to reflect whether the 'Card' or 'Cash' payment methods.

In [12]:
renaming_dict_payment_type = {'Adult Card Fare Per Ride' : 'Card',
       'Persons With Disabilities Card Fare Per Ride' : 'Card',
       'Senior Citizen Card Fare Per Ride' : 'Card', 
       'Student Card Fare Per Ride' : 'Card',
       'Workfare Transport Concession Card Fare Per Ride' : 'Card',
       'Adult Cash Fare Per Ride' : 'Cash',
       'Persons With Disabilities Cash Fare Per Ride' : 'Cash',
       'Senior Citizen Cash Fare Per Ride' : 'Cash', 
       'Student Cash Fare Per Ride' : 'Cash',
       'Workfare Transport Concession Cash Fare Per Ride' : 'Cash',
       'Cash': 'Cash',
       'Card': 'Card'}

trunks2['Payment Type'] = trunks2['Payment Type'].map(renaming_dict_payment_type)

print(trunks2.head())

print(trunks2.tail())

  Passenger Type Payment Type  Fare Distance (Km)
0          Adult         Card    83   3.2 or less
1          Adult         Card    93    3.3  - 4.2
2          Adult         Card   103    4.3  - 5.2
3          Adult         Card   113    5.3  - 6.2
4          Adult         Card   122    6.3  - 7.2
    Passenger Type Payment Type  Fare Distance (Km)
385         Worker         Cash   260  36.3  - 37.2
386         Worker         Cash   260  37.3  - 38.2
387         Worker         Cash   260  38.3  - 39.2
388         Worker         Cash   260  39.3  - 40.2
389         Worker         Cash   260  40.2 or more


In [13]:
trunks2['Payment Type'].value_counts()

Card    195
Cash    195
Name: Payment Type, dtype: int64

### Cleaning the dataframe for Express Buses

The same cleaning processes were applied to the dataframe for express buses.

First, the dataframe was read in, and unnecessary columns were discarded.

In [14]:
express2 = pd.read_csv('express_pivot.csv')


express2.drop(['F1', 'Number of Records'], axis=1, inplace=True)
express2.head()

,Passenger Type,Payment Type,Pivot Field Values,Distance (Km)
0,Adult,Adult Card Fare Per Ride,143,3.2 or less
1,Adult,Adult Card Fare Per Ride,153,3.3 - 4.2
2,Adult,Adult Card Fare Per Ride,163,4.3 - 5.2
3,Adult,Adult Card Fare Per Ride,173,5.3 - 6.2
4,Adult,Adult Card Fare Per Ride,182,6.3 - 7.2


Next, the column values in `Passenger Type` were renamed. 

In [15]:
rename_dict2 = {'Adult': 'Adult',
'Persons': 'Disabled',
'Senior': 'Senior',
'Student': 'Student',
'Workfare': 'Worker',
'Cash': 'All',
'Disabled': 'Disabled',
'Worker': 'Worker'
'All': 'All'}

express2['Passenger Type'] = express2['Passenger Type'].map(rename_dict2)

express2['Passenger Type'].value_counts()

Student     39
All         39
Senior      39
Adult       39
Worker      39
Disabled    39
Name: Passenger Type, dtype: int64

After verifying that the column values were replaced correctly, the `Payment Type` column's values were then changed to reflect 'Card' or 'Cash' payment options.

In [16]:
renaming_dict_payment_type_2 = {'Cash Fare Per Ride':'Cash',
       'Adult Card Fare Per Ride' : 'Card',
       'Persons With Disabilities Card Fare Per Ride' : 'Card',
       'Senior Citizen Card Fare Per Ride' : 'Card', 
       'Student Card Fare Per Ride' : 'Card',
       'Workfare Transport Concession Card Fare Per Ride' : 'Card',
       'Cash': 'Cash',
       'Card': 'Card'
       }

express2['Payment Type'] = express2['Payment Type'].map(renaming_dict_payment_type_2)

express2['Payment Type'].value_counts()

Card    195
Cash     39
Name: Payment Type, dtype: int64

## Concatenating the two dataframes

After verifying that the cleaning was all done, we'll add a new column in the dataframe to represent the Type of Bus (Trunk Bus or Express Bus), and then we'll preview both dataframes a final time.

In [27]:
express2['Transport'] = "Express Bus"

print(express2.head())

trunks2['Transport'] = "Trunk Bus"

print(trunks2.head())

  Passenger Type Payment Type  Fare Distance (Km)    Transport
0          Adult         Card   143   3.2 or less  Express Bus
1          Adult         Card   153    3.3  - 4.2  Express Bus
2          Adult         Card   163    4.3  - 5.2  Express Bus
3          Adult         Card   173    5.3  - 6.2  Express Bus
4          Adult         Card   182    6.3  - 7.2  Express Bus
  Passenger Type Payment Type  Fare Distance (Km)  Transport
0          Adult         Card    83   3.2 or less  Trunk Bus
1          Adult         Card    93    3.3  - 4.2  Trunk Bus
2          Adult         Card   103    4.3  - 5.2  Trunk Bus
3          Adult         Card   113    5.3  - 6.2  Trunk Bus
4          Adult         Card   122    6.3  - 7.2  Trunk Bus


In [18]:
# express2 = express2.rename({'Pivot Field Values':'Fare'}, axis=1)
# express2.head()

,Passenger Type,Payment Type,Fare,Distance (Km),Transport
0,Adult,Card,143,3.2 or less,Express Bus
1,Adult,Card,153,3.3 - 4.2,Express Bus
2,Adult,Card,163,4.3 - 5.2,Express Bus
3,Adult,Card,173,5.3 - 6.2,Express Bus
4,Adult,Card,182,6.3 - 7.2,Express Bus


Let's confirm the sizes of both dataframes, and then concatenate them:

In [19]:
express2.shape

(234, 5)

In [20]:
trunks2.shape

(390, 5)

In [29]:
combined = pd.concat([express2, trunks2])

print(combined.shape)

print(combined.head())
print(combined.tail())

(624, 5)
  Passenger Type Payment Type  Fare Distance (Km)    Transport
0          Adult         Card   143   3.2 or less  Express Bus
1          Adult         Card   153    3.3  - 4.2  Express Bus
2          Adult         Card   163    4.3  - 5.2  Express Bus
3          Adult         Card   173    5.3  - 6.2  Express Bus
4          Adult         Card   182    6.3  - 7.2  Express Bus
    Passenger Type Payment Type  Fare Distance (Km)  Transport
385         Worker         Cash   260  36.3  - 37.2  Trunk Bus
386         Worker         Cash   260  37.3  - 38.2  Trunk Bus
387         Worker         Cash   260  38.3  - 39.2  Trunk Bus
388         Worker         Cash   260  39.3  - 40.2  Trunk Bus
389         Worker         Cash   260  40.2 or more  Trunk Bus


## Exporting back to Tableau

After verifying that the concatenation was done correctly, we'll finall export the dataframes to Tableau.

In [23]:
combined.to_csv('trunk_and_express')